<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

# Prepare an RGI shapefile from a list of IDs

In this notebook, we demonstrate how to make a subset of the large RGI files based on a list of IDs (here, taken from the Leclerq database of glacier lengths)

In [ ]:
import os
from glob import glob
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from oggm import utils, cfg
cfg.initialize()
%matplotlib inline

### Read the Leclerq link file

In [ ]:
df = pd.read_csv(utils.get_demo_file('rgi_leclercq_links_2012_RGIV5.csv'))

Let's add a column with the region's name, this is useful for later:

In [ ]:
df['RGI_REG'] = [n.split('.')[0].split('-')[1] for n in df.RGI_ID]

### Read the shapefiles per region

In [ ]:
rgi_dir = utils.get_rgi_dir()

Now loop over teh regions and take the galciers we want (this can take some time!):

In [ ]:
selection = []
for reg in df.RGI_REG.unique():
    fs = list(sorted(glob(os.path.join(rgi_dir, '*', '*' + reg + '_rgi50_*.shp'))))[0]
    sh = gpd.read_file(fs)
    selection.append(sh.loc[sh.RGIId.isin(df.RGI_ID)])
# Make a new dataframe of those
selection = pd.concat(selection)
selection.csr = sh.crs  # for geolocalisation

### Plot and store 

In [ ]:
selection.plot();

In [ ]:
# This stores the file in your home
selection.to_file(os.path.join(os.path.expanduser('~'), 'rgi_selection.shp'))